In [ ]:
from pathlib import Path

root_results_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences"
# Define the directory to search
directory = Path(root_results_path)

# Use rglob to search recursively ('**/*.csv') 
# or glob for just the current folder ('*.csv')
csv_files = []
for path in list(directory.rglob('*.csv')):
    if "tests" in str(path) or len(str(path).split('/')[-1])<20:
        continue
    csv_files.append(path)

root_results_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences_postprocess"
# Define the directory to search
directory = Path(root_results_path)

# Use rglob to search recursively ('**/*.csv') 
# or glob for just the current folder ('*.csv')
for path in list(directory.rglob('*.csv')):
    if "tests" in str(path) or len(str(path).split('/')[-1])<20:
        continue
    csv_files.append(path)

csv_files

In [ ]:
import pandas as pd
import glob
import os
from datetime import datetime

# Show all rows
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)
# Set options to show all rows and columns
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None) # Prevents text inside cells from being cut off


rows_list = []
cols_to_remove = ['tif_paths', 'id', 'pred_paths']

for file in csv_files:
    filename = os.path.basename(file)
    c_time = os.path.getctime(file)
    dt_c_time = datetime.fromtimestamp(c_time)
    df = pd.read_csv(file)
    
    if len(df) >= 80:
        mean_row = df.iloc[[79]].copy()
        clean_row = mean_row.drop(columns=cols_to_remove, errors='ignore')
        
        # --- PARSING FILENAME ---
        clean_row['Losses'] = filename.split('_DA')[0] if '_DA' in filename else  filename.split('_epoch')[0]
        clean_row['epoch'] = filename.split('epoch_')[-1].split('_')[0] 
        clean_row['Train DA'] = filename.split('_DA')[-1].split('_')[0] if '_DA' in filename else  filename.split('_epoch')[0][-4:]
        clean_row['tta_status'] = 'no_TTA' if 'no_TTA' in filename else 'TTA'
        
        if 'overlap_' in filename:
            overlap_val = filename.split('overlap_')[1].split('_')[0]
            clean_row['overlap'] = float(overlap_val)
            
        if 'th_' in filename:
            th_val = filename.split('th_')[1].replace('.csv', '')
            clean_row['threshold'] = float(th_val)
        clean_row['dt_c_time'] = dt_c_time

        if 'post_process_' in str(file).split('/')[-2]:
            clean_row['postprocess'] = str(file).split('/')[-2].split('post_process_')[-1]
        else:
            clean_row['postprocess'] = 'no_postprocess'
        rows_list.append(clean_row)

# 2. Create the new combined DataFrame
combined_df = pd.concat(rows_list, ignore_index=True)

# 3. REORDER COLUMNS: Define the leading order
# We list exactly what we want first, then add the rest of the columns
fixed_cols = ['Losses', 'epoch', 'Train DA', 'tta_status', 'overlap', 'threshold', 'postprocess']
other_cols = [c for c in combined_df.columns if c not in fixed_cols]

combined_df = combined_df[fixed_cols + other_cols]

# 4. View the result
combined_df.sort_values(['image_score'], ascending=False)
#combined_df.sort_values(['Losses', 'epoch', 'tta_status', 'image_score'], ascending=False)

In [ ]:

import pandas as pd

# Basic save
combined_df.to_csv('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/resume_results_old.csv', index=False)

# check individual results

In [ ]:
df_1 = pd.read_csv("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/BCE_Tversky_DAsafe_epoch_model_best_TTA_overlap_0.5_th_0.5.csv")
df_1.sort_values("topo_score", ascending=False)

In [ ]:
df_2 = pd.read_csv(
    "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated/results_post.csv")
df_2[df['id'] == "2961547523"]

In [ ]:
# Merge the two dataframes on the 'id' column
combined_df = df_1.merge(df_2[['id', 'voi_score']], on='id', suffixes=('_1', '_2'))

combined_df['diff'] = combined_df['voi_score_1'] - combined_df['voi_score_2']
combined_df = combined_df.sort_values('diff', ascending=False)
combined_df = combined_df[['id', 'voi_score_1', 'voi_score_2', 'diff']]
print(f"mean: {combined_df['diff'].mean()}")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(combined_df)



# Copy all csv to an independent folder

In [ ]:
from pathlib import Path
import shutil

# Define directories
base_path = Path("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences")
dest_root = Path("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results")

# Find all .csv files
csv_files = list(base_path.rglob("*.csv"))

for file in csv_files:
    file_str = str(file)
    
    # Filtering logic
    if '/tests/' in file_str or file.name.startswith('th_'):
        continue
    
    # Create the new path using pathlib (cleaner than .replace)
    relative_path = file.relative_to(base_path)
    dest_path = dest_root / relative_path

    # Create the folder structure in the destination root
    dest_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Copy the file
    shutil.copy2(file, dest_path)
    print(f"Copied: {dest_path}")